In [14]:
# Mid Level A uzduočiai alikti, (vizualizuoti Lietuvos savivaldybes pagal jų antrinio sergamumo (prevalence) 
# rangą padieniui didžiausio sergamumo 30 dienų laikotarpiu) buvo susikelti iš Agreguoti_COVID19.xlsx duomenys.   
# Gyventojų skaičiaus kitimas padieniui nėra skelbiamas, skelbiama kas mėnesį, todėl buvo naudojami gimusių ir mirusių 
# duomenys. Nustačius kad dižiausi susirgimų skaičiai yra 2022 metų vasarį, šiam mėnesiui buvo paskaičiuotas
# gyventojų skaičiaus kitimas padieniui, panaudojant interpoliaciją. Imigracijos duomenys į skaičiavimus nebuvo įtraukti.

In [ ]:
import pandas as pd

# susikeliami duomenys iš Excelio filų,į duomenų matricas visas gyventojų skaičius (df_visi), gimusių (df_gime) ir mirusių (df_mire) per pirmus 2022 metų mėnesius.
df_mire = pd.read_excel('/Users/daivadaugelaite/Downloads/data-mire2.xlsx')
df_gime = pd.read_excel('/Users/daivadaugelaite/Downloads/data-gime2.xlsx')   
df_visi = pd.read_excel('/Users/daivadaugelaite/Downloads/data-visi2.xlsx')

In [39]:

# sujungiama duomenų matrica (df_visi) su duomenų matrica df_gime pagal savivaldybių ('Municipalities') indeksą. 
aligned_df = df_visi.join(df_gime, lsuffix='_visi', rsuffix='_2022M01').fillna(0)

# prijungiama duomenų matrica df_mire su mirusiųjų duomenimis
aligned_df = aligned_df.join(df_mire, lsuffix='_gyvi', rsuffix='_mire').fillna(0)

In [41]:
print(aligned_df)
aligned_df.to_excel("gyvent_matrica.xlsx", index=False)

    Municipalities_visi       Visi    Miestas    Kaimas  \
0   Lietuvos Respublika  2805998.0  1913385.0  892613.0   
1     Sostinės regionas   817950.0   651145.0  166805.0   
2    Vilniaus apskritis   817950.0   651145.0  166805.0   
3        Elektrėnų sav.    23521.0    15678.0    7843.0   
4    Šalčininkų r. sav.    29699.0    10594.0   19105.0   
..                  ...        ...        ...       ...   
69    Ignalinos r. sav.    14464.0     5703.0    8761.0   
70       Molėtų r. sav.    16926.0     5722.0   11204.0   
71       Utenos r. sav.    37266.0    25204.0   12062.0   
72        Visagino sav.    19404.0    19214.0     190.0   
73       Zarasų r. sav.    14790.0     6442.0    8348.0   

   Municipalities_2022M01  2022M01_gyvi  2022M02_gyvi  2022M03_gyvi  \
0     Lietuvos Respublika        1799.0        1601.0        1826.0   
1       Sostinės regionas         631.0         567.0         620.0   
2      Vilniaus apskritis         631.0         567.0         620.0   
3      

In [16]:

# Apply the first calculation
aligned_df_first_calculation = aligned_df.copy()  # kopija duomenų matricos
aligned_df_first_calculation.loc[:, 'Visi'] += (aligned_df_first_calculation.loc[:, '2022M02_gyvi'] + aligned_df_first_calculation.loc[:, '2022M01_existing']
                                               - aligned_df_first_calculation.loc[:, '2022M01_mire'] - aligned_df_first_calculation.loc[:, '2022M02_mire'])


In [17]:

# Apply the second calculation
aligned_df_second_calculation = aligned_df.copy()
aligned_df_second_calculation.loc[:, 'Visi'] += (aligned_df_second_calculation.loc[:, '2022M02_gyvi'] + aligned_df_second_calculation.loc[:, '2022M01_gyvi'] + aligned_df_second_calculation.loc[:, '2022M03_gyvi']
                                                 - aligned_df_second_calculation.loc[:, '2022M01_mire'] - aligned_df_second_calculation.loc[:, '2022M02_mire'] - aligned_df_second_calculation.loc[:, '2022M03_mire'])


In [18]:
# Calculate the difference
aligned_df_difference = aligned_df_second_calculation.copy()
aligned_df_difference.loc[:, 'Visi_Difference'] = (aligned_df_second_calculation.loc[:, 'Visi'] - aligned_df_first_calculation.loc[:, 'Visi']) / 28


In [19]:
# sukurima tuščia duomenų matrica 28 dienom, 
new_df = pd.DataFrame(index=pd.RangeIndex(1, 74))

In [20]:

new_df.reset_index(drop=True, inplace=True)

In [21]:
# print(aligned_df)

    Municipalities_visi       Visi    Miestas    Kaimas  \
0   Lietuvos Respublika  2805998.0  1913385.0  892613.0   
1     Sostinės regionas   817950.0   651145.0  166805.0   
2    Vilniaus apskritis   817950.0   651145.0  166805.0   
3        Elektrėnų sav.    23521.0    15678.0    7843.0   
4    Šalčininkų r. sav.    29699.0    10594.0   19105.0   
..                  ...        ...        ...       ...   
69    Ignalinos r. sav.    14464.0     5703.0    8761.0   
70       Molėtų r. sav.    16926.0     5722.0   11204.0   
71       Utenos r. sav.    37266.0    25204.0   12062.0   
72        Visagino sav.    19404.0    19214.0     190.0   
73       Zarasų r. sav.    14790.0     6442.0    8348.0   

   Municipalities_2022M01  2022M01_existing  2022M02_existing  \
0     Lietuvos Respublika            1799.0            1601.0   
1       Sostinės regionas             631.0             567.0   
2      Vilniaus apskritis             631.0             567.0   
3          Elektrėnų sav.      

In [22]:
# pridedama pirmo stulpelio pavadinimas "Municipalities" 
new_df['Municipalities'] = aligned_df_difference['Municipalities']

In [23]:
# pridedama stulpelio pavadinimas 'Visi' 
new_df['Visi'] = aligned_df_first_calculation['Visi']

In [24]:
# pridedamas stulpelis duomenų "Visi_Difference"  
new_df['Visi_Difference'] = aligned_df_difference['Visi_Difference']

In [25]:
# pridedami 28 nauji stulepliai Vasario mėnesio dienom 
for day in reversed(range(1, 29)):
    new_df.insert(loc=1, column=f'Day_{day}', value=None)  # Nauji stulpeliai įterpiami nuo antros pozicijos, pirma skirta savivaldybių pavadinimam.


In [28]:
print(new_df)
new_df.to_excel("new_df.xlsx")

         Municipalities          Day_1          Day_2          Day_3  \
0   Lietuvos Respublika      2801124.0      2801045.0      2800966.0   
1     Sostinės regionas  817125.678571  817112.357143  817099.035714   
2    Vilniaus apskritis  817125.678571  817112.357143  817099.035714   
3        Elektrėnų sav.   23488.392857   23487.785714   23487.178571   
4    Šalčininkų r. sav.   29619.214286   29618.428571   29617.642857   
..                  ...            ...            ...            ...   
68     Anykščių r. sav.   22709.214286   22707.428571   22705.642857   
69    Ignalinos r. sav.        14406.5        14405.0        14403.5   
70       Molėtų r. sav.   16869.142857   16868.285714   16867.428571   
71       Utenos r. sav.        37175.5        37174.0        37172.5   
72        Visagino sav.   19359.357143   19358.714286   19358.071429   

            Day_4          Day_5          Day_6      Day_7          Day_8  \
0       2800887.0      2800808.0      2800729.0  2800650.0

In [30]:
print(new_df.columns)

Index(['Municipalities', 'Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6',
       'Day_7', 'Day_8', 'Day_9', 'Day_10', 'Day_11', 'Day_12', 'Day_13',
       'Day_14', 'Day_15', 'Day_16', 'Day_17', 'Day_18', 'Day_19', 'Day_20',
       'Day_21', 'Day_22', 'Day_23', 'Day_24', 'Day_25', 'Day_26', 'Day_27',
       'Day_28', 'Visi', 'Visi_Difference'],
      dtype='object')


In [31]:
#  suskaičiuoja, interpoliuoja, kiek gyventojų kasdieną buvo vasario mėnesį, 

for index, row in new_df.iterrows():
    # Iterate through each day column in new_df, excluding the last two columns
    for day_column in new_df.columns[:-2]:  # paskutinius du stulpelius praleidžia ('Visi', 'Visi_Difference')
        # tikrina ar  stulpelio pavadinimas prasideda 'Day_'
        if not day_column.startswith('Day_'):
            print(f"Skipping column '{day_column}' due to naming inconsistency.")
            continue
        
        #  dienos skaičius iš stulpelio pavadinimo 
        try:
            day_number = int(day_column.split('_')[1])
        except IndexError:
            print(f"Skipping column '{day_column}' due to naming inconsistency.")
            continue

        # paskaičiuojama esamos dienos vertė, žmonių skaičius 
        if day_number == 1:
            # Day 1,  'Visi' + 'Visi_Difference'
            new_df.at[index, day_column] = row['Visi'] + row['Visi_Difference']
        else:
            # kitom mėnesio dienom, 'Visi' +  'Visi_Difference' * dienų skaičiaus 
            new_df.at[index, day_column] = row['Visi'] + (day_number * row['Visi_Difference'])


Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping column 'Municipalities' due to naming inconsistency.
Skipping

In [33]:
# išsaugojama duomenų matrica į xlsx
new_df.to_excel("updated_new_df.xlsx", index=False)
print(new_df)

         Municipalities          Day_1          Day_2          Day_3  \
0   Lietuvos Respublika      2801124.0      2801045.0      2800966.0   
1     Sostinės regionas  817125.678571  817112.357143  817099.035714   
2    Vilniaus apskritis  817125.678571  817112.357143  817099.035714   
3        Elektrėnų sav.   23488.392857   23487.785714   23487.178571   
4    Šalčininkų r. sav.   29619.214286   29618.428571   29617.642857   
..                  ...            ...            ...            ...   
68     Anykščių r. sav.   22709.214286   22707.428571   22705.642857   
69    Ignalinos r. sav.        14406.5        14405.0        14403.5   
70       Molėtų r. sav.   16869.142857   16868.285714   16867.428571   
71       Utenos r. sav.        37175.5        37174.0        37172.5   
72        Visagino sav.   19359.357143   19358.714286   19358.071429   

            Day_4          Day_5          Day_6      Day_7          Day_8  \
0       2800887.0      2800808.0      2800729.0  2800650.0

In [34]:
# new_df pervadinama į population_df ir išsaugojama xlsx formatu
population_df = new_df.copy(deep=True)
population_df.to_excel("population_df.xlsx", index=False)